In [39]:
from webscrapeData import updateData

updateData('2025-26')
database_year = "2526"

In [19]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error as MSE

In [20]:
df = pd.read_csv('/Users/amanshaik/Documents/GitHub/PlayerGaugeAI/database/merged.csv')
df['SALARY'] = df['SALARY'].replace('[\$,]', '', regex=True).astype(float)

df.dropna(subset='SALARY',inplace=True)
X = df.drop('SALARY', axis=1)
y = df['SALARY']
y = y.replace('[\$,]', '', regex=True).astype(float)




In [21]:
df.head()
df['PPG'] = round(df['PTS']/df['GP'],1)
df['RPG'] = round(df['REB']/df['GP'],1)
df['APG'] = round(df['AST']/df['GP'],1)



Feature Importance

In [22]:
columns_to_drop = ['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION']
X.drop(columns=[col for col in columns_to_drop if col in X.columns], inplace=True)

df_corr = pd.concat([X, y], axis=1)

# Compute Spearman correlation
spearman_corr = df_corr.corr(method='spearman')

corr_df = spearman_corr[['SALARY']].reset_index().rename(columns={'index': 'Feature'})

corr_df['ABS_CORR'] = abs(corr_df['SALARY'])
corr_df.sort_values(by = 'ABS_CORR', ascending=False, inplace=True)
corr_df.head(30)

,Feature,SALARY,ABS_CORR
61,SALARY,1.000000,1.000000
25,PTS,0.623405,0.623405
55,PTS_RANK,-0.623405,0.623405
36,FGM_RANK,-0.622615,0.622615
6,FGM,0.622615,0.622615
37,FGA_RANK,-0.616314,0.616314
7,FGA,0.616314,0.616314
60,WNBA_FANTASY_PTS_RANK,-0.614987,0.614987
30,WNBA_FANTASY_PTS,0.614987,0.614987
27,NBA_FANTASY_PTS,0.610239,0.610239


In [23]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import numpy as np
import pandas as pd

corr_df = spearman_corr[['SALARY']].reset_index().rename(columns={'index': 'Feature'})
corr_df['ABS_CORR'] = abs(corr_df['SALARY'])
corr_df.sort_values(by='ABS_CORR', ascending=False, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    max_depth=4,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

# Train
model.fit(X_train, y_train)

# Predict & evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(MSE(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

# Create importance DataFrame
importance_df = pd.DataFrame({
    'Feature': model.feature_names_in_,
    'Importance': model.feature_importances_
})
importance_df.sort_values(by='Importance', ascending=False, inplace=True)


importance_filtered = importance_df[importance_df['Importance'] > 0.01]

corr_imp_table = importance_df.merge(corr_df, on='Feature')
corr_imp_table.sort_values(by = ['Importance','ABS_CORR'],inplace=True, ascending=False)
feat_cut_df = corr_imp_table[corr_imp_table['Feature'].str.contains('RANK')]
feat_cut_list = feat_cut_list = feat_cut_df['Feature'].tolist()
print(feat_cut_list)



RMSE: 7824161.16
['NBA_FANTASY_PTS_RANK', 'PTS_RANK', 'FGA_RANK', 'PLUS_MINUS_RANK', 'MIN_RANK', 'FG3_PCT_RANK', 'FG_PCT_RANK', 'FGM_RANK', 'WNBA_FANTASY_PTS_RANK', 'FTM_RANK', 'PFD_RANK', 'FTA_RANK', 'TOV_RANK', 'AST_RANK', 'DREB_RANK', 'BLKA_RANK', 'STL_RANK', 'REB_RANK', 'DD2_RANK', 'FG3A_RANK', 'FG3M_RANK', 'PF_RANK', 'BLK_RANK', 'OREB_RANK', 'W_RANK', 'GP_RANK', 'FT_PCT_RANK', 'TD3_RANK', 'L_RANK', 'W_PCT_RANK']


In [24]:
# Initialize model with parameters
feat_cut_list.append('AGE')
feat_cut_list.append('GP')
model_cut = xgb.XGBRegressor(
    objective='reg:squarederror',
    max_depth=4,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

# Train
model_cut.fit(X_train[feat_cut_list], y_train)

# Predict & evaluate
y_pred = model_cut.predict(X_test[feat_cut_list])
rmse = np.sqrt(MSE(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")


RMSE: 7934578.15


In [25]:
df['PREDICTED_SALARY'] = model_cut.predict(df[feat_cut_list])
top_pred_df = df.sort_values(by = 'PREDICTED_SALARY',ascending=False)
top_pred_df.head(15)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SALARY,PPG,RPG,APG,PREDICTED_SALARY
501,201939,Stephen Curry,Stephen,1610612744,GSW,37.0,70,41,29,0.586,...,43,19,71,44,13,55761216.0,24.5,4.4,6.0,53951804.0
277,203954,Joel Embiid,Joel,1610612755,PHI,31.0,19,8,11,0.421,...,390,289,87,20,287,51415938.0,23.8,8.2,4.5,50544676.0
424,203999,Nikola Jokić,Nikola,1610612743,DEN,30.0,70,46,24,0.657,...,2,1,2,1,1,51415938.0,29.6,12.7,10.2,50310576.0
330,201142,Kevin Durant,Kevin,1610612756,PHX,36.0,62,33,29,0.532,...,361,30,106,44,24,51179021.0,26.6,6.0,4.2,49743656.0
180,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,30.0,67,40,27,0.597,...,28,3,5,2,4,48787676.0,30.4,11.9,6.5,49068100.0
314,1626157,Karl-Anthony Towns,Karl-Anthony,1610612752,NYK,29.0,72,46,26,0.639,...,21,8,4,20,9,49205800.0,24.4,12.8,3.1,48924916.0
359,2544,LeBron James,LeBron,1610612747,LAL,40.0,70,44,26,0.629,...,383,10,17,3,10,48728845.0,24.4,7.8,8.2,47428808.0
141,1626164,Devin Booker,Devin,1610612756,PHX,28.0,75,35,40,0.467,...,507,14,35,44,11,49205800.0,25.6,4.1,7.1,47362224.0
51,203078,Bradley Beal,Bradley,1610612756,PHX,32.0,53,22,31,0.415,...,537,138,198,44,127,50203930.0,17.0,3.3,3.7,46634556.0
255,1627759,Jaylen Brown,Jaylen,1610612738,BOS,28.0,63,46,17,0.730,...,18,54,106,44,52,49700000.0,22.2,5.8,4.5,46444640.0


In [26]:
df['SALARY_DIFF'] = df['PREDICTED_SALARY'] - df['SALARY']
df['SALARY_PCT_CHANGE'] = ((df['PREDICTED_SALARY'] - df['SALARY'])/df['SALARY'])*100
# improved_df = df[df['SALARY_DIFF']>0]
# improved_df.sort_values(by = 'SALARY_DIFF',ascending=False)

df.sort_values(by = 'SALARY_DIFF',ascending=True)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SALARY,PPG,RPG,APG,PREDICTED_SALARY,SALARY_DIFF,SALARY_PCT_CHANGE
41,1627732,Ben Simmons,Ben,1610612746,LAC,28.0,51,26,25,0.510,...,198,44,261,40338144.0,5.0,4.7,5.6,15043256.0,-25294888.0,-62.707119
112,203081,Damian Lillard,Damian,1610612749,MIL,34.0,58,32,26,0.552,...,40,15,41,48787676.0,24.9,4.7,7.1,28912910.0,-19874766.0,-40.737267
316,202695,Kawhi Leonard,Kawhi,1610612746,LAC,34.0,37,26,11,0.703,...,106,44,169,49205800.0,21.5,5.9,3.1,32017928.0,-17187872.0,-34.930581
546,1630169,Tyrese Haliburton,Tyrese,1610612754,IND,25.0,73,46,27,0.630,...,18,44,19,42176400.0,18.6,3.5,9.2,25411114.0,-16765286.0,-39.750396
63,1628971,Bruce Brown,Bruce,1610612740,NOP,28.0,41,15,26,0.366,...,281,44,299,23000000.0,8.3,4.0,2.0,6495753.5,-16504246.5,-71.757593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,1630208,Nick Richards,Nick,1610612756,PHX,27.0,57,17,40,0.298,...,35,44,203,5000000.0,9.3,8.2,0.9,16556462.0,11556462.0,231.129240
185,1627824,Guerschon Yabusele,Guerschon,1610612755,PHI,29.0,70,23,47,0.329,...,134,44,122,2087519.0,11.0,5.6,2.1,18774070.0,16686551.0,799.348461
498,203915,Spencer Dinwiddie,Spencer,1610612742,DAL,32.0,79,37,42,0.468,...,106,44,108,2087519.0,11.0,2.6,4.4,21373112.0,19285593.0,923.852334
36,1630559,Austin Reaves,Austin,1610612747,LAL,27.0,73,46,27,0.630,...,71,20,30,12976362.0,20.2,4.5,5.8,34417920.0,21441558.0,165.235511


*New Model*

In [27]:
df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
       'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
       'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
       'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
       'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
       'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK', 'SALARY', 'PPG', 'RPG',
       'APG', 'PREDICTED_SALARY', 'SALARY_DIFF', 'SALARY_PCT_CHANGE'],
      dtype='object')

In [28]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import numpy as np
import pandas as pd

# Load dataset
df = pd.read_csv('../database/merged.csv')

# Clean salary column
df['SALARY'] = df['SALARY'].replace('[\$,]', '', regex=True).astype(float)

# Create condensed salary in millions
df['SALARY_CONDENSED'] = df['SALARY'].apply(lambda x: round(x / 1_000_000, 2))

# Drop rows with missing or invalid salary values
df = df.dropna(subset=['SALARY', 'SALARY_CONDENSED'])
df = df[~df['SALARY_CONDENSED'].isin([np.inf, -np.inf])]

# Define features and target
features = ['NBA_FANTASY_PTS_RANK', 'PTS_RANK', 'FGA_RANK', 'PLUS_MINUS_RANK', 'MIN_RANK',
            'FG3_PCT_RANK', 'FG_PCT_RANK', 'FGM_RANK', 'WNBA_FANTASY_PTS_RANK', 'FTM_RANK',
            'PFD_RANK', 'FTA_RANK', 'TOV_RANK', 'AST_RANK', 'DREB_RANK', 'BLKA_RANK',
            'STL_RANK', 'REB_RANK', 'DD2_RANK', 'FG3A_RANK', 'FG3M_RANK', 'PF_RANK',
            'BLK_RANK', 'OREB_RANK', 'W_RANK', 'GP_RANK', 'FT_PCT_RANK', 'TD3_RANK',
            'L_RANK', 'W_PCT_RANK']

X = df[features]
y = df['SALARY_CONDENSED']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train XGBoost model
new_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    max_depth=4,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

new_model.fit(X_train, y_train)

# Evaluate
y_pred = new_model.predict(X_test)
rmse = np.sqrt(MSE(y_test, y_pred))
print(f"RMSE (in millions): {rmse:.2f}")

# Predict on full dataset
df['PREDICTED_SALARY'] = new_model.predict(X)
df['PPG'] = round(df['PTS']/df['GP'],1)
df['RPG'] = round(df['REB']/df['GP'],1)
df['APG'] = round(df['AST']/df['GP'],1)

df['SALARY_DIFF'] = df['PREDICTED_SALARY'] - df['SALARY_CONDENSED']
df['SALARY_PCT_CHANGE'] = ((df['PREDICTED_SALARY'] - df['SALARY_CONDENSED'])/df['SALARY_CONDENSED'])*100

# Show top 15 highest predicted salaries
top_pred_df = df.sort_values(by='PREDICTED_SALARY', ascending=False)
top_pred_df.head(15)[['PLAYER_NAME', 'PREDICTED_SALARY','SALARY_PCT_CHANGE']].reset_index()


RMSE (in millions): 8.77


,index,PLAYER_NAME,PREDICTED_SALARY,SALARY_PCT_CHANGE
0,501,Stephen Curry,53.259766,-4.483921
1,424,Nikola Jokić,50.449413,-1.887567
2,277,Joel Embiid,50.062534,-2.639957
3,330,Kevin Durant,49.628387,-3.031678
4,180,Giannis Antetokounmpo,48.530663,-0.531538
5,359,LeBron James,48.494038,-0.484224
6,314,Karl-Anthony Towns,48.351124,-1.745329
7,141,Devin Booker,46.954796,-4.582817
8,451,Paul George,46.919876,-4.653777
9,255,Jaylen Brown,45.738274,-7.971280


In [29]:
import plotly.express as px
top_13_per_team = (
    df.sort_values(by=['TEAM_ABBREVIATION', 'PREDICTED_SALARY'], ascending=[True, False])
      .groupby('TEAM_ABBREVIATION')
      .head(13)
)
wealthiest_df = top_13_per_team[['TEAM_ABBREVIATION','PLAYER_NAME','PREDICTED_SALARY']]
team_salary_totals = (
    wealthiest_df
    .groupby('TEAM_ABBREVIATION')['PREDICTED_SALARY']
    .sum()
    .reset_index()
    .sort_values(by='PREDICTED_SALARY', ascending=False)
)
wealthiest_teams = team_salary_totals.sort_values(by="PREDICTED_SALARY", ascending=True).head(5)
poorest_teams = team_salary_totals.sort_values(by="PREDICTED_SALARY", ascending=False).head(5)


In [30]:
import pandas as pd

player_info_df = pd.read_csv("df_players.csv")

# Step 1: Make all column names in player_info_df uppercase
player_info_df.columns = [col.upper() for col in player_info_df.columns]

# Step 2: Create PLAYER_NAME column in player_info_df to match df
player_info_df['PLAYER_NAME'] = (player_info_df['FIRST_NAME'] + ' ' + player_info_df['LAST_NAME']).str.strip()

# Step 3: Drop FIRST_NAME and LAST_NAME if you no longer need them
# (optional, only if they’re not useful post-merge)
# player_info_df = player_info_df.drop(columns=['FIRST_NAME', 'LAST_NAME'])

# Step 4: Merge
merged_df = df.merge(player_info_df, how='left', on='PLAYER_NAME')

# Step 5: Preview
merged_df.drop(['TEAM_ID_y'], axis=1, inplace=True)
merged_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID_x',
       'TEAM_ABBREVIATION', 'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD',
       'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3',
       'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK', 'SALARY', 'SALARY_CONDENSED',
       'PREDICTED_SALARY', 'PPG', 'RPG', 'APG', 'SALARY_DIFF',
       'SALARY_PCT_CHANGE', 'UNNAMED: 0', 'ID', 'FIRST_NAME', 'LAST_NAME',
       'POS

In [31]:
import sqlite3

# Step 1: Create a connection to a SQLite database file (it creates the file if it doesn't exist)
conn = sqlite3.connect('nba_data'+database_year+'.db')

# Step 2: Write the DataFrame to a SQL table
merged_df.to_sql('player_data', conn, if_exists='replace', index=False)

# Step 3: (Optional) Close the connection when done
conn.close()


In [32]:
import plotly.express as px

# Top 5 Wealthiest Teams - Vertical Bar Chart
fig_wealthiest = px.bar(
    poorest_teams.sort_values(by="PREDICTED_SALARY", ascending=False),
    x='TEAM_ABBREVIATION',
    y='PREDICTED_SALARY',
    title='Top 5 Wealthiest Teams (Predicted Salary Total)',
    labels={'PREDICTED_SALARY': 'Total Salary (Millions)', 'TEAM_ABBREVIATION': 'Team'},
    color='PREDICTED_SALARY',
    color_continuous_scale='Blues'
)
fig_wealthiest.update_layout(xaxis=dict(categoryorder='total descending'))
fig_wealthiest.show()
fig_wealthiest.write_html('../database/wealthiestTeams.html')

# Top 5 Poorest Teams - Vertical Bar Chart
fig_poorest = px.bar(
    wealthiest_teams.sort_values(by="PREDICTED_SALARY", ascending=True),
    x='TEAM_ABBREVIATION',
    y='PREDICTED_SALARY',
    title='Top 5 Poorest Teams (Predicted Salary Total)',
    labels={'PREDICTED_SALARY': 'Total Salary (Millions)', 'TEAM_ABBREVIATION': 'Team'},
    color='PREDICTED_SALARY',
    color_continuous_scale='Reds'
)
fig_poorest.update_layout(xaxis=dict(categoryorder='total ascending'))
fig_poorest.show()
fig_poorest.write_html('../database/poorestTeams.html')


In [33]:
# import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go

# Full NBA team logos dictionary (ESPN 500px logos)
team_logos = {
    "ATL": "https://a.espncdn.com/i/teamlogos/nba/500/atl.png",
    "BOS": "https://a.espncdn.com/i/teamlogos/nba/500/bos.png",
    "BKN": "https://a.espncdn.com/i/teamlogos/nba/500/bkn.png",
    "CHA": "https://a.espncdn.com/i/teamlogos/nba/500/cha.png",
    "CHI": "https://a.espncdn.com/i/teamlogos/nba/500/chi.png",
    "CLE": "https://a.espncdn.com/i/teamlogos/nba/500/cle.png",
    "DAL": "https://a.espncdn.com/i/teamlogos/nba/500/dal.png",
    "DEN": "https://a.espncdn.com/i/teamlogos/nba/500/den.png",
    "DET": "https://a.espncdn.com/i/teamlogos/nba/500/det.png",
    "GSW": "https://a.espncdn.com/i/teamlogos/nba/500/gs.png",
    "HOU": "https://a.espncdn.com/i/teamlogos/nba/500/hou.png",
    "IND": "https://a.espncdn.com/i/teamlogos/nba/500/ind.png",
    "LAC": "https://a.espncdn.com/i/teamlogos/nba/500/lac.png",
    "LAL": "https://a.espncdn.com/i/teamlogos/nba/500/lal.png",
    "MEM": "https://a.espncdn.com/i/teamlogos/nba/500/mem.png",
    "MIA": "https://a.espncdn.com/i/teamlogos/nba/500/mia.png",
    "MIL": "https://a.espncdn.com/i/teamlogos/nba/500/mil.png",
    "MIN": "https://a.espncdn.com/i/teamlogos/nba/500/min.png",
    "NOP": "https://a.espncdn.com/i/teamlogos/nba/500/no.png",
    "NYK": "https://a.espncdn.com/i/teamlogos/nba/500/ny.png",
    "OKC": "https://a.espncdn.com/i/teamlogos/nba/500/okc.png",
    "ORL": "https://a.espncdn.com/i/teamlogos/nba/500/orl.png",
    "PHI": "https://a.espncdn.com/i/teamlogos/nba/500/phi.png",
    "PHX": "https://a.espncdn.com/i/teamlogos/nba/500/phx.png",
    "POR": "https://a.espncdn.com/i/teamlogos/nba/500/por.png",
    "SAC": "https://a.espncdn.com/i/teamlogos/nba/500/sac.png",
    "SAS": "https://a.espncdn.com/i/teamlogos/nba/500/sa.png",
    "TOR": "https://a.espncdn.com/i/teamlogos/nba/500/tor.png",
    "UTA": "https://a.espncdn.com/i/teamlogos/nba/500/utah.png",
    "WAS": "https://a.espncdn.com/i/teamlogos/nba/500/wsh.png",
}

# ===== Wealthiest Teams Chart =====
teams_wealthiest = poorest_teams.sort_values(by="PREDICTED_SALARY", ascending=False)

fig_wealthiest = go.Figure()

fig_wealthiest.add_trace(
    go.Bar(
        x=teams_wealthiest['TEAM_ABBREVIATION'],
        y=teams_wealthiest['PREDICTED_SALARY'],
        marker=dict(color=teams_wealthiest['PREDICTED_SALARY'], colorscale="Blues"),
        text=[
            f"<img src='{team_logos[t]}' style='height:40px'>" if t in team_logos else ""
            for t in teams_wealthiest['TEAM_ABBREVIATION']
        ],
        textposition="outside"
    )
)

fig_wealthiest.update_layout(
    title="Top 5 Wealthiest Teams (Predicted Salary Total)",
    xaxis=dict(title="Team", categoryorder='total descending'),
    yaxis=dict(title="Total Salary (Millions)")
)

fig_wealthiest.show()
fig_wealthiest.write_html('../database/wealthiestTeams.html')

# ===== Poorest Teams Chart =====
teams_poorest = wealthiest_teams.sort_values(by="PREDICTED_SALARY", ascending=True)

fig_poorest = go.Figure()

fig_poorest.add_trace(
    go.Bar(
        x=teams_poorest['TEAM_ABBREVIATION'],
        y=teams_poorest['PREDICTED_SALARY'],
        marker=dict(color=teams_poorest['PREDICTED_SALARY'], colorscale="Reds"),
        text=[
            f"<img src='{team_logos[t]}' style='height:40px'>" if t in team_logos else ""
            for t in teams_poorest['TEAM_ABBREVIATION']
        ],
        textposition="outside"
    )
)

fig_poorest.update_layout(
    title="Top 5 Poorest Teams (Predicted Salary Total)",
    xaxis=dict(title="Team", categoryorder='total ascending'),
    yaxis=dict(title="Total Salary (Millions)")
)

fig_poorest.show()
fig_poorest.write_html('../database/poorestTeams.html')


In [34]:
import plotly.express as px
# Full NBA team logos dictionary (ESPN 500px logos)
team_logos = {
    "ATL": "https://a.espncdn.com/i/teamlogos/nba/500/atl.png",
    "BOS": "https://a.espncdn.com/i/teamlogos/nba/500/bos.png",
    "BKN": "https://a.espncdn.com/i/teamlogos/nba/500/bkn.png",
    "CHA": "https://a.espncdn.com/i/teamlogos/nba/500/cha.png",
    "CHI": "https://a.espncdn.com/i/teamlogos/nba/500/chi.png",
    "CLE": "https://a.espncdn.com/i/teamlogos/nba/500/cle.png",
    "DAL": "https://a.espncdn.com/i/teamlogos/nba/500/dal.png",
    "DEN": "https://a.espncdn.com/i/teamlogos/nba/500/den.png",
    "DET": "https://a.espncdn.com/i/teamlogos/nba/500/det.png",
    "GSW": "https://a.espncdn.com/i/teamlogos/nba/500/gs.png",
    "HOU": "https://a.espncdn.com/i/teamlogos/nba/500/hou.png",
    "IND": "https://a.espncdn.com/i/teamlogos/nba/500/ind.png",
    "LAC": "https://a.espncdn.com/i/teamlogos/nba/500/lac.png",
    "LAL": "https://a.espncdn.com/i/teamlogos/nba/500/lal.png",
    "MEM": "https://a.espncdn.com/i/teamlogos/nba/500/mem.png",
    "MIA": "https://a.espncdn.com/i/teamlogos/nba/500/mia.png",
    "MIL": "https://a.espncdn.com/i/teamlogos/nba/500/mil.png",
    "MIN": "https://a.espncdn.com/i/teamlogos/nba/500/min.png",
    "NOP": "https://a.espncdn.com/i/teamlogos/nba/500/no.png",
    "NYK": "https://a.espncdn.com/i/teamlogos/nba/500/ny.png",
    "OKC": "https://a.espncdn.com/i/teamlogos/nba/500/okc.png",
    "ORL": "https://a.espncdn.com/i/teamlogos/nba/500/orl.png",
    "PHI": "https://a.espncdn.com/i/teamlogos/nba/500/phi.png",
    "PHX": "https://a.espncdn.com/i/teamlogos/nba/500/phx.png",
    "POR": "https://a.espncdn.com/i/teamlogos/nba/500/por.png",
    "SAC": "https://a.espncdn.com/i/teamlogos/nba/500/sac.png",
    "SAS": "https://a.espncdn.com/i/teamlogos/nba/500/sa.png",
    "TOR": "https://a.espncdn.com/i/teamlogos/nba/500/tor.png",
    "UTA": "https://a.espncdn.com/i/teamlogos/nba/500/utah.png",
    "WAS": "https://a.espncdn.com/i/teamlogos/nba/500/wsh.png",
}

teams_wealthiest = poorest_teams.sort_values(by="PREDICTED_SALARY", ascending=False)


fig = px.bar(
    teams_wealthiest,
    x='TEAM_ABBREVIATION',
    y='PREDICTED_SALARY',
    color='PREDICTED_SALARY',
    color_continuous_scale='Blues',
    labels={'PREDICTED_SALARY': 'Total Salary (Millions)', 'TEAM_ABBREVIATION': 'Team'},
    title="Top 5 Wealthiest Teams (Predicted Salary Total)"
)

fig.update_layout(
    xaxis=dict(categoryorder='total descending'),
    width=800,   # square width
    height=600   # square height
)

for i, row in teams_wealthiest.iterrows():
    team = row['TEAM_ABBREVIATION']
    salary = row['PREDICTED_SALARY']
    if team in team_logos:
        fig.add_layout_image(
            dict(
                source=team_logos[team],
                x=team,
                y=salary / 2,        # center vertically inside the bar
                xref="x",
                yref="y",
                xanchor="center",     # center horizontally
                yanchor="middle",     # align middle of image with y
                sizex=0.3,            # fraction of category width
                sizey=salary * 0.5,   # scale relative to bar height
                layer="above"
            )
        )

fig.show()
fig.write_html('../database/wealthiestTeams2.html')
import plotly.express as px

teams_poorest = wealthiest_teams.sort_values(by="PREDICTED_SALARY", ascending=True)

fig = px.bar(
    teams_poorest,
    x='TEAM_ABBREVIATION',
    y='PREDICTED_SALARY',
    color='PREDICTED_SALARY',
    color_continuous_scale='Reds',
    labels={'PREDICTED_SALARY': 'Total Salary (Millions)', 'TEAM_ABBREVIATION': 'Team'},
    title="Top 5 Poorest Teams (Predicted Salary Total)"
)

# fig.update_layout(xaxis=dict(categoryorder='total ascending'))
fig.update_layout(
    xaxis=dict(categoryorder='total ascending'),
    width=800,   # square width
    height=600   # square height
)
for i, row in teams_poorest.iterrows():
    team = row['TEAM_ABBREVIATION']
    salary = row['PREDICTED_SALARY']
    if team in team_logos:
        fig.add_layout_image(
            dict(
                source=team_logos[team],
                x=team,
                y=salary / 2,        # center vertically inside the bar
                xref="x",
                yref="y",
                xanchor="center",     # center horizontally
                yanchor="middle",     # align middle of image with y
                sizex=0.3,            # fraction of category width
                sizey=salary * 0.5,   # scale relative to bar height
                layer="above"
            )
        )

fig.show()

fig.write_html('../database/poorestTeams2.html')



In [35]:
import plotly.express as px

# Wealthiest Teams
teams_wealthiest = poorest_teams.sort_values(by="PREDICTED_SALARY", ascending=False)

fig = px.bar(
    teams_wealthiest,
    x='TEAM_ABBREVIATION',
    y='PREDICTED_SALARY',
    color='PREDICTED_SALARY',
    color_continuous_scale='Blues',
    labels={'PREDICTED_SALARY': 'Total Salary (Millions)', 'TEAM_ABBREVIATION': 'Team'},
    title="Top 5 Wealthiest Teams (Predicted Salary Total)"
)

fig.update_layout(
    xaxis=dict(categoryorder='total descending'),
    width=600,   # set width
    height=600   # set height to make it square
)

fig.show()
fig.write_html('../database/wealthiestTeams2.html')


# Poorest Teams
teams_poorest = wealthiest_teams.sort_values(by="PREDICTED_SALARY", ascending=True)

fig = px.bar(
    teams_poorest,
    x='TEAM_ABBREVIATION',
    y='PREDICTED_SALARY',
    color='PREDICTED_SALARY',
    color_continuous_scale='Reds',
    labels={'PREDICTED_SALARY': 'Total Salary (Millions)', 'TEAM_ABBREVIATION': 'Team'},
    title="Top 5 Poorest Teams (Predicted Salary Total)"
)

fig.update_layout(
    xaxis=dict(categoryorder='total ascending'),
    width=600,   # square width
    height=600   # square height
)

fig.show()
fig.write_html('../database/poorestTeams2.html')


In [36]:
# !pip install -U kaleido

In [37]:

import plotly.express as px

teams_poorest = wealthiest_teams.sort_values(by="PREDICTED_SALARY", ascending=True)

fig = px.bar(
    teams_poorest,
    x='TEAM_ABBREVIATION',
    y='PREDICTED_SALARY',
    color='PREDICTED_SALARY',
    color_continuous_scale='Reds',
    labels={'PREDICTED_SALARY': 'Team Salary (Millions)', 'TEAM_ABBREVIATION': 'Team'},
    title="Top 5 Poorest Teams (Predicted Salary Total)"
)

fig.update_layout(xaxis=dict(categoryorder='total ascending'))

for i, row in teams_poorest.iterrows():
    team = row['TEAM_ABBREVIATION']
    salary = row['PREDICTED_SALARY']
    if team in team_logos:
        fig.add_layout_image(
            dict(
                source=team_logos[team],
                x=team,
                y=salary / 2,        # center vertically inside the bar
                xref="x",
                yref="y",
                xanchor="center",     # center horizontally
                yanchor="middle",     # align middle of image with y
                sizex=0.3,            # fraction of category width
                sizey=salary * 0.5,   # scale relative to bar height
                layer="above"
            )
        )

fig.show()
fig.write_image(
    "../database/poorestTeams.png",  # path and file type
    width=1200,     # width in pixels
    height=800,     # height in pixels
    scale=2         # 2x resolution (makes it sharp)
)



ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
import plotly.express as px

# Step 1: Get top 13 paid players per team
top_13_age = (
    df.sort_values(by=['TEAM_ABBREVIATION', 'PREDICTED_SALARY'], ascending=[True, False])
      .groupby('TEAM_ABBREVIATION')
      .head(13)
)

# Step 2: Compute average age per team
avg_age_df = (
    top_13_age.groupby('TEAM_ABBREVIATION')['AGE']
    .mean()
    .reset_index()
    .sort_values(by='AGE', ascending=False)
)

# Step 3: Plot vertical bar chart
fig_avg_age = px.bar(
    avg_age_df,
    x='TEAM_ABBREVIATION',
    y='AGE',
    title='Average Age of Top 13 Predicted Salary Players per Team',
    labels={'AGE': 'Average Age', 'TEAM_ABBREVIATION': 'Team'},
    color='AGE',
    color_continuous_scale='Viridis'
)
fig_avg_age.update_layout(xaxis=dict(categoryorder='total descending'))
fig_avg_age.show()


In [ ]:
import plotly.express as px

# Map percent change to colors based on your rules
def map_color(change):
    if change > 5:
        return "green"
    elif change < -5:
        return "red"
    else:
        return "yellow"

df["COLOR"] = df["SALARY_PCT_CHANGE"].apply(map_color)

# Bubble plot
fig = px.scatter(
    df,
    x="AGE",
    y="PREDICTED_SALARY",
    size="PPG",   # bubble size ~ salary magnitude
    color="COLOR",
    color_discrete_map={"green": "green", "yellow": "yellow", "red": "red"},
    hover_data=["PLAYER_NAME", "SALARY_CONDENSED", "PREDICTED_SALARY", "SALARY_PCT_CHANGE"]
)

fig.update_layout(
    title="Age vs Predicted Salary (Bubble Plot)",
    xaxis_title="Age",
    yaxis_title="Predicted Salary (Millions)",
    legend_title="Salary Change Category"
)

fig.show()
